# System Identification - FOPDT Model

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import skopt
from skopt.space import Real, Integer
from problems.optprob.problems import ( 
    solve_problem_with_optimizer,
    solve_problem_with_optimizer_n_repeats
)
from problems.optprob.plot_utils import (
    function_evaluations_plot, best_guesses_plot, best_guesses_plot_n_repeats
)
from problems.sys_id_fopdt import (
    make_simulate_function,
    rms_prediction_error,
    SysIdFOPDT, 
    calculate_reasonable_bounds
)

import lpfgopt
lpfgopt.__version__

## Load Input-Output Dataset

In [ ]:
data_dir = 'data'
plot_dir = 'plots'
os.makedirs(plot_dir, exist_ok=True)
os.listdir(data_dir)

In [ ]:
filename = 'io_data_fopdt.csv'
input_output_data = pd.read_csv(os.path.join(data_dir, filename))
input_output_data

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(7, 5.5))

data = input_output_data.set_index('Time')

for ax, name in zip(axes, data.columns):
    data[name].plot(ax=ax, grid=True, title=name)

plt.tight_layout()
plt.show()

## Construct Simulation Function

In [ ]:
input_col = 'Input1'
t = input_output_data['Time'].to_numpy()
u_data = input_output_data[input_col].to_numpy()
y_data = input_output_data['Output'].to_numpy()

# Determine average time interval
time_step_sizes = np.diff(t)
dt = np.mean(time_step_sizes)
assert np.max(np.abs(time_step_sizes - dt)) < dt / 10
dt

In [ ]:
# Make simulation function
simulate = make_simulate_function(dt, u_data)

[-11.89612273, 360.83577464, 500.66886236, 534.13787627]

# Test simulate function
K = -10.0
tau = 400.0
n_delay = 2  # must be integer 
y_base = 500.0
u_base = 5.0
y_init = 530.0

y_model = simulate(K, tau, n_delay, y_base, u_base, y_init)
rms_prediction_error(y_model[n_delay+1:], y_data[n_delay+1:])

In [ ]:
plt.figure(figsize=(7, 2.5))
plt.plot(t, input_output_data["Output"].to_numpy(), label='data')
plt.plot(t, y_model, label='y_model')
plt.xlabel('Time')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
%%timeit

y_model = simulate(K, tau, n_delay, y_base, u_base, y_init)

## Construct Optimization Problem Class

In [ ]:
bounds = calculate_reasonable_bounds(t, u_data, y_data)
var_names = list(bounds.keys())
bounds = list(bounds.values())
bounds

In [ ]:
# TODO: Include option to specify initial guess ranges
# Low	High
# [-12	-8]
# [150	200]
# [150	250]
# [470	500]
# 5	5  # Note: span is zero!  This is because the problem is overspecified.  I.e. one variable is redundant.  (Could have chosen y_base or y_init)
# [470	490]

In [ ]:
problem = SysIdFOPDT(bounds, dt, u_data, y_data)

# Test cost function evaluation
x = [K, tau, n_delay, y_base, y_init]
rms_error = problem.cost_function_to_minimize(x, u_base=u_base)
rms_error

In [ ]:
# Global minimum
K, tau, n_delay, y_base, y_init = [-11.89612273, 360.83577464, 8, 500.66886236, 534.13787627]
x_global_minimum = K, tau, n_delay, y_base, y_init
print(problem.cost_function_to_minimize(x_global_minimum))

In [ ]:
y_model = simulate(K, tau, n_delay, y_base, u_base, y_init)

plt.figure(figsize=(7, 2.5))
plt.plot(t, input_output_data["Output"].to_numpy(), label='data')
plt.plot(t, y_model, label='y_model')
plt.xlabel('Time')
plt.grid()
plt.legend()
plt.title('Global Best Solution')
plt.tight_layout()
plt.show()

In [ ]:
sol = solve_problem_with_optimizer(
    problem, 
    lpfgopt.minimize, 
    problem.bounds,
    discrete=[2],
    points=50,
    tol=0.1,
    maxit=1000,
    seedval=0
)
sol

In [ ]:
function_evaluations_plot(problem)
plt.tight_layout()
plt.show()

In [ ]:
best_guesses_plot(problem)
plt.tight_layout()
plt.show()

In [ ]:
n_repeats = 100
fun_evals, unique_solutions, best_guesses = solve_problem_with_optimizer_n_repeats(
    problem, 
    lpfgopt.minimize, 
    n_repeats, 
    problem.bounds,
    discrete=[2],
    points=50,
    tol=0.01,
    maxit=1000,
)
ax = best_guesses_plot_n_repeats(fun_evals)
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, f"lf_convergence_plot_{n_repeats}.png"), dpi=150)
plt.show()

In [ ]:
min(best_guesses)

In [ ]:
x0 = [-11.895919407570858, 360.8357737663191, 8, 500.66909556507204, 534.1378658992475]

f_final = problem(x0)

assert f_final == 5.043253589891017

# Fix n_delay to current value and reduce problem
f5 = lambda x: problem([x[0], x[1], 8, x[2], x[3]])

x0 = [-11.895919407570858, 360.8357737663191, 500.66909556507204, 534.1378658992475]

assert f5(x0) == f_final

In [ ]:
# Do additional gradient descent at this point.
res = scipy.optimize.minimize(f5, x0=x0, tol=0.0001)
assert res.status == 0
res.fun, res.x

## Bayesian Optimization

In [ ]:
bounds

In [ ]:
# Define dimensions - these should be the same as bounds above
dimensions = [
    Real(np.float64(-85.37499999999989), np.float64(85.37499999999989), transform='normalize'),
    Real(np.float64(15.02181818181818), np.float64(4406.4), transform='normalize'),
    Integer(0, 88),  # Integers don't need/support transform parameter
    Real(np.float64(390.72511918274705), np.float64(609.2851191827467), transform='normalize'),
    Real(np.float64(474.79), np.float64(584.0699999999999), transform='normalize')
]

dimensions

In [ ]:
# Run Bayesian optimization
# 500 iterations takes ___ mins
problem.reset()
res = skopt.gp_minimize(
    problem,
    dimensions,
    n_calls=250,
    noise=1e-10,
    random_state=0,
    n_initial_points=20,
    verbose=True
)
res

In [ ]:
res['x'], res['fun']

In [ ]:
problem.best_guess

In [ ]:
function_evaluations_plot(problem)
plt.tight_layout()
plt.show()

In [ ]:
best_guesses_plot(problem)
plt.tight_layout()
plt.show()

In [ ]:
n_repeats = 10
fun_evals, unique_solutions, best_guesses = solve_problem_with_optimizer_n_repeats(
    problem, 
    skopt.gp_minimize, 
    n_repeats,
    dimensions,
    noise=1e-10,
    n_calls=250,
    n_initial_points=20,
)
unique_solutions

In [ ]:
ax = best_guesses_plot_n_repeats(fun_evals)
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, f"lf_convergence_plot_{n_repeats}.png"), dpi=150)
plt.show()

In [ ]:
min(best_guesses)

In [ ]:
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# Save results
np.save(os.path.join(results_dir, "fun_evals.npy"), np.stack(fun_evals))
best_guesses_x = np.stack([item[1] for item in best_guesses])
assert best_guesses_x.shape == (10, 6)
np.save(os.path.join(results_dir, "best_guesses_x.npy"), best_guesses_x)
best_guesses_fun = np.stack([item[0] for item in best_guesses])
assert best_guesses_fun.shape == (10,)
np.save(os.path.join(results_dir, "best_guesses_fun.npy"), best_guesses_fun)